In [11]:
import stanza
import pandas as pd
import sqlite3

In [13]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')

articles = pd.read_json('articles_with_sentiment.json')
articles.head()

2022-03-17 15:19:23 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2022-03-17 15:19:23 INFO: Use device: cpu
2022-03-17 15:19:23 INFO: Loading: tokenize
2022-03-17 15:19:23 INFO: Loading: sentiment
2022-03-17 15:19:23 INFO: Done loading processors!


,title,text,href,date,text_size,HKUST_title,HKUST_text,PROSUS_title,PROSUS_text
1496,Brown-Forman stock slips after profit misses o...,Shares of Jack Daniels parent Brown-Forman Cor...,https://www.marketwatch.com/story/brown-forman...,2020-12-08,153,-0.999502,0.985009,-0.973591,-0.973772
1498,This analysis of Wall Street stock ratings is ...,"In the financial media, “Wall Street” typicall...",https://www.marketwatch.com/story/this-analysi...,2020-09-07,272,0.000000,0.000000,0.000000,0.000000
1499,These 74 stocks in the S&P 500 hit all-time re...,U.S. stocks staged a broad rally Wednesday as ...,https://www.marketwatch.com/story/these-74-sto...,2020-09-03,206,0.991664,0.000000,0.794795,0.000000
1500,"Brown-Forman's stock slips, although earnings ...",Shares of Jack Daniel's parent Brown-Forman Co...,https://www.marketwatch.com/story/brown-forman...,2020-09-02,192,-0.999107,0.956672,-0.959359,-0.940531
1502,"Brown-Forman's stock gains after sales beat, a...","Shares of Brown-Forman Corp. BF.A,\n+0.38%\ned...",https://www.marketwatch.com/story/brown-forman...,2020-06-09,190,1.000000,-0.898049,0.954016,-0.970586


In [85]:
def extract_sentiment(text):
    doc = nlp(text)
    if len(doc.sentences) == 1:
        return doc.sentences[0].sentiment - 1
    else:
        sentiment = 0
        for sentence in doc.sentences:
            sentiment += sentence.sentiment
        return sentiment / len(doc.sentences) - 1

In [92]:
extract_sentiment('earnings are amazing. sales are ok.')

0.5

In [94]:
articles['STANZA_title'] = articles['title'].apply(extract_sentiment)

In [95]:
articles['STANZA_text'] = articles['text'].apply(extract_sentiment)

In [96]:
articles.to_json('new_articles.json')

In [104]:
articles.iloc[:,5:].corr().style.background_gradient(cmap='coolwarm')

,HKUST_title,HKUST_text,PROSUS_title,PROSUS_text,STANZA_title,STANZA_text
HKUST_title,1.000000,0.534960,0.727255,0.307935,0.401917,0.323098
HKUST_text,0.534960,1.000000,0.638755,0.542050,0.508276,0.372331
PROSUS_title,0.727255,0.638755,1.000000,0.433470,0.498305,0.265003
PROSUS_text,0.307935,0.542050,0.433470,1.000000,0.317451,0.289973
STANZA_title,0.401917,0.508276,0.498305,0.317451,1.000000,0.320410
STANZA_text,0.323098,0.372331,0.265003,0.289973,0.320410,1.000000
